In [2]:
pip install tqdm --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 732.3 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 2.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
import torch
import torchvision
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, Subset, ConcatDataset
import numpy as np
import copy
from tqdm import tqdm

# Function to create a globally shared dataset
def create_shared_dataset(full_dataset, share_percent=0.05):
    shared_size = int(len(full_dataset) * share_percent)
    indices = np.random.choice(range(len(full_dataset)), size=shared_size, replace=False)
    shared_dataset = torch.utils.data.Subset(full_dataset, indices)
    return shared_dataset

# Function to split the dataset into clients
def split_dataset_into_clients(dataset, num_clients=10):
    total_size = len(dataset)
    per_client = total_size // num_clients
    client_indices = [range(i * per_client, (i + 1) * per_client) for i in range(num_clients)]
    # Adjust the last client's range to include any leftover elements
    client_indices[-1] = range((num_clients - 1) * per_client, total_size)
    return client_indices

# Simple perceptron model definition
class SimplePerceptron(torch.nn.Module):
    def __init__(self):
        super(SimplePerceptron, self).__init__()
        self.flatten = torch.nn.Flatten()
        self.linear = torch.nn.Linear(28*28, 10)  # 28x28 pixels to 10 classes

    def forward(self, x):
        x = self.flatten(x)
        x = self.linear(x)
        return x

# Function to train local model
def train_local_model(model, device, train_loader, optimizer, epoch, mu, global_weights):
    model.train()
    criterion = torch.nn.CrossEntropyLoss()
    for batch_idx, (data, target) in enumerate(tqdm(train_loader)):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        
        # Adding proximal term
        proximal_term = 0.0
        for param_key, param in model.named_parameters():
            global_param = global_weights[param_key]
            proximal_term += (mu / 2) * torch.norm(param - global_param) ** 2
        loss += proximal_term
        
        loss.backward()
        optimizer.step()

# Function to evaluate model
def test_model(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    criterion = torch.nn.CrossEntropyLoss(reduction='sum')
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.0f}%)')

# Main federated training function
def fedprox_share_train(global_model, device, train_dataset, test_loader, epochs=1, mu=0.01, num_clients=10, frac=0.1):
    global_weights = global_model.state_dict()
    client_indices = split_dataset_into_clients(train_dataset, num_clients)
    shared_dataset = create_shared_dataset(train_dataset)  # Create shared dataset

    for epoch in range(epochs):
        local_weights = []
        m = max(int(frac * num_clients), 1)
        selected_clients = np.random.choice(range(num_clients), m, replace=False)
        
        for client in selected_clients:
            local_model = copy.deepcopy(global_model)
            local_model.to(device)
            optimizer = torch.optim.SGD(local_model.parameters(), lr=0.01)
            
            local_dataset = Subset(train_dataset, list(client_indices[client]))
            combined_dataset = ConcatDataset([local_dataset, shared_dataset])
            train_loader = DataLoader(combined_dataset, batch_size=64, shuffle=True)
            
            train_local_model(local_model, device, train_loader, optimizer, epoch, mu, global_weights)
            local_weights.append(local_model.state_dict())
        
        # Average local models
        global_weights = {key: torch.stack([local_weights[i][key] for i in range(len(local_weights))]).mean(0) for key in global_weights.keys()}
        global_model.load_state_dict(global_weights)
        
        test_model(global_model, device, test_loader)

# Main execution
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load MNIST dataset
    train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=ToTensor())
    test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=ToTensor())
    test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

    global_model = SimplePerceptron().to(device)

    fedprox_share_train(global_model, device, train_dataset, test_loader, epochs=20, mu=0.01, num_clients=100, frac=0.1)


100%|██████████████████████████████████████████████████████████████████████████████| 57/57 [00:00<00:00, 201.90it/s]


Test set: Average loss: 1.8123, Accuracy: 6979/10000 (70%)


100%|██████████████████████████████████████████████████████████████████████████████| 57/57 [00:00<00:00, 211.59it/s]


Test set: Average loss: 1.4836, Accuracy: 7792/10000 (78%)


100%|██████████████████████████████████████████████████████████████████████████████| 57/57 [00:00<00:00, 205.54it/s]


...

Test set: Average loss: 0.5851, Accuracy: 8694/10000 (87%)


100%|██████████████████████████████████████████████████████████████████████████████| 57/57 [00:00<00:00, 201.44it/s]


Test set: Average loss: 0.5737, Accuracy: 8705/10000 (87%)


100%|██████████████████████████████████████████████████████████████████████████████| 57/57 [00:00<00:00, 208.83it/s]


Test set: Average loss: 0.5636, Accuracy: 8711/10000 (87%)
